In [10]:
import pandas as pd
#d300 = pd.read_json('/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/NER_RESUME-300.json', lines=True)
d700 = pd.read_json('/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/pre_trained-700.json', lines=True)

In [4]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f: #reading
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation'] #columns name annonatation cases

        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

d300 = trim_entity_spans(convert_dataturks_to_spacy("/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/NER_RESUME-300.json")) #300
#d700 = trim_entity_spans(convert_dataturks_to_spacy("/Users/cosmos/Desktop/DeepBlue/BACKEND/ML/Datasets (UNCLEANED)/pre_trained-700.json")) #700

In [6]:
d300[1]

['Afreen Jamadar Active member of IIIT Committee in Third year  Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6  I wish to use my knowledge, skills and conceptual understanding to create excellent team environments and work consistently achieving organization objectives believes in taking initiative and work to excellence in my work.  WORK EXPERIENCE  Active member of IIIT Committee in Third year  Cisco Networking -  Kanpur, Uttar Pradesh  organized by Techkriti IIT Kanpur and Azure Skynet. PERSONALLITY TRAITS: • Quick learning ability • hard working  EDUCATION  PG-DAC  CDAC ACTS  2017  Bachelor of Engg in Information Technology  Shivaji University Kolhapur -  Kolhapur, Maharashtra  2016  SKILLS  Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)  ADDITIONAL INFORMATION  TECHNICAL SKILLS:  • Programming Languages: C, C++, Java, .net, php. • Web 

In [19]:
d700['annotation'][0]

[{'label': ['Email Address'],
  'points': [{'start': 1155,
    'end': 1198,
    'text': 'indeed.com/r/Afreen-Jamadar/8baf379b705e37c6'}]},
 {'label': ['Links'],
  'points': [{'start': 1143,
    'end': 1239,
    'text': 'https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN'}]},
 {'label': ['Skills'],
  'points': [{'start': 743,
    'end': 1140,
    'text': 'Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming Languages: C, C++, Java, .net, php.\n• Web Designing: HTML, XML\n• Operating Systems: Windows […] Windows Server 2003, Linux.\n• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 729, 'end': 732, 'text': '2016'}]},
 {'label': ['College Name'],
  'points': [{'start': 675,
    'end': 702,
    'text': 'Shiva